# import required libraries and read dataset



In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
creditCardData = pd.read_csv('Credit_card.csv')
CreditCardLabel=pd.read_csv('Credit_card_label.csv')
assert len(creditCardData)==len(CreditCardLabel)
# By default, merge() performs an inner join based on columns with the same name in both datasets.
creditCardData=creditCardData.merge(CreditCardLabel)
creditCardData.tail()

,Ind_ID,GENDER,Car_Owner,Propert_Owner,CHILDREN,Annual_income,Type_Income,EDUCATION,Marital_status,Housing_type,Birthday_count,Employed_days,Mobile_phone,Work_Phone,Phone,EMAIL_ID,Type_Occupation,Family_Members,label
1543,5028645,F,N,Y,0,NaN,Commercial associate,Higher education,Married,House / apartment,-11957.0,-2182,1,0,0,0,Managers,2,0
1544,5023655,F,N,N,0,225000.0,Commercial associate,Incomplete higher,Single / not married,House / apartment,-10229.0,-1209,1,0,0,0,Accountants,1,0
1545,5115992,M,Y,Y,2,180000.0,Working,Higher education,Married,House / apartment,-13174.0,-2477,1,0,0,0,Managers,4,0
1546,5118219,M,Y,N,0,270000.0,Working,Secondary / secondary special,Civil marriage,House / apartment,-15292.0,-645,1,1,1,0,Drivers,2,0
1547,5053790,F,Y,Y,0,225000.0,Working,Higher education,Married,House / apartment,-16601.0,-2859,1,0,0,0,NaN,2,0


# Data preprocessing:


In [16]:
# How many rows have missing values?
creditCardData.isnull().sum()

Ind_ID               0
GENDER               7
Car_Owner            0
Propert_Owner        0
CHILDREN             0
Annual_income       23
Type_Income          0
EDUCATION            0
Marital_status       0
Housing_type         0
Birthday_count      22
Employed_days        0
Mobile_phone         0
Work_Phone           0
Phone                0
EMAIL_ID             0
Type_Occupation    488
Family_Members       0
label                0
dtype: int64

In [3]:
# # Delete all rows where Occupation is missing:
# creditCardData.dropna(subset=['Type_Occupation'],inplace=True)
# data where Annual_income is missing:

annualMissing=creditCardData[creditCardData['Annual_income'].isnull()]
#  Type_Occupation against thee missing values
Type_Occupation=annualMissing['Type_Occupation'].value_counts().to_frame().reset_index()
Type_Occupation.columns=['Type_Occupation','Count']
Type_Occupation=Type_Occupation['Type_Occupation'].tolist()
print(Type_Occupation)
#  Find the mean of all Occupations in Type_Occupation existing  in the credit card data:
meanOccupation=creditCardData.groupby('Type_Occupation')['Annual_income'].mean().to_frame().reset_index()
#  all the Annual_income in meanOccupation existing  in Type_Occupation:
meanOccupation=meanOccupation[meanOccupation['Type_Occupation'].isin(Type_Occupation)]
# impute Annual_income in annualMissing with corresponding meanOccupation:


for index,row in annualMissing.iterrows():
    for index1,row1 in meanOccupation.iterrows():
        if row['Type_Occupation']==row1['Type_Occupation']:
            annualMissing.loc[index,'Annual_income']=row1['Annual_income']
# save the data in dictionary making Type_Occupation as key and Annual_income rounded to two figures as value:
annualMissing=annualMissing[['Type_Occupation','Annual_income']]
annualMissing=annualMissing.set_index('Type_Occupation')
annualMissing=annualMissing.to_dict()['Annual_income']
# impute the missing values in creditCardData with the values in annualMissing:
creditCardData['Annual_income']=creditCardData['Annual_income'].fillna(creditCardData['Type_Occupation'].map(annualMissing))

# annualMissing[['Type_Occupation','Annual_income']]
# Merge the two DataFrames based on 'Type_Occupation'
# creditCardData=creditCardData.merge(meanOccupation,how='left',on='Type_Occupation')


['Managers', 'Cooking staff', 'Laborers', 'Cleaning staff', 'High skill tech staff', 'Sales staff', 'Accountants', 'Security staff', 'Core staff', 'Drivers']


In [4]:
creditCardData.isnull().sum()

Ind_ID               0
GENDER               7
Car_Owner            0
Propert_Owner        0
CHILDREN             0
Annual_income        8
Type_Income          0
EDUCATION            0
Marital_status       0
Housing_type         0
Birthday_count      22
Employed_days        0
Mobile_phone         0
Work_Phone           0
Phone                0
EMAIL_ID             0
Type_Occupation    488
Family_Members       0
label                0
dtype: int64

In [5]:
# Display Ind_ID, Type_Occupation, Annual_income, Type_Occupation where Annual_income is missing:
creditCardData[creditCardData['Annual_income'].isnull()][['Ind_ID','Type_Occupation','Annual_income']]


,Ind_ID,Type_Occupation,Annual_income
3,5009749,NaN,NaN
27,5024917,NaN,NaN
65,5058348,NaN,NaN
125,5116305,NaN,NaN
273,5047704,NaN,NaN
561,5024636,NaN,NaN
915,5143620,NaN,NaN
1263,5092147,NaN,NaN


In [6]:
# drop irrelavant columns

col = ['Ind_ID', 'Mobile_phone', 'Work_Phone', 'EMAIL_ID', 'Phone']
creditCardData = creditCardData.drop(columns=col)
creditCardData.head()

,GENDER,Car_Owner,Propert_Owner,CHILDREN,Annual_income,Type_Income,EDUCATION,Marital_status,Housing_type,Birthday_count,Employed_days,Type_Occupation,Family_Members,label
0,M,Y,Y,0,180000.0,Pensioner,Higher education,Married,House / apartment,-18772.0,365243,NaN,2,1
1,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,-13557.0,-586,NaN,2,1
2,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,NaN,-586,NaN,2,1
3,F,Y,N,0,NaN,Commercial associate,Higher education,Married,House / apartment,-13557.0,-586,NaN,2,1
4,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,-13557.0,-586,NaN,2,1


In [17]:
# make column head all lowercase

creditCardData.columns = [x.lower() for x in creditCardData.columns]
creditCardData.head(5)

,ind_id,gender,car_owner,propert_owner,children,annual_income,type_income,education,marital_status,housing_type,birthday_count,employed_days,mobile_phone,work_phone,phone,email_id,type_occupation,family_members,label
0,5008827,M,Y,Y,0,180000.0,Pensioner,Higher education,Married,House / apartment,-18772.0,365243,1,0,0,0,NaN,2,1
1,5009744,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,-13557.0,-586,1,1,1,0,NaN,2,1
2,5009746,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,NaN,-586,1,1,1,0,NaN,2,1
3,5009749,F,Y,N,0,NaN,Commercial associate,Higher education,Married,House / apartment,-13557.0,-586,1,1,1,0,NaN,2,1
4,5009752,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,-13557.0,-586,1,1,1,0,NaN,2,1


In [19]:
# # Change Age type.
# # F = 1
# # M = 0
# creditCardData.gender = [1 if i == "F" else 0 for i in creditCardData.gender]
# # Change BP(Blood Pressure Levels) type.
# # LOW = 2 , Normal = 1 , High = 0
# creditCardData.car_owner = [1 if i== "Y" else 0 for i in creditCardData.car_owner]
# creditCardData.propert_owner = [1 if i== "Y" else 0 for i in creditCardData.car_owner]
# # Change Cholesterol type.
# # HIGH = 1 , NORMAL = 0
yes_no_categorical_columns=['car_owner',  'propert_owner']

yes_no_values={
1:'yes', 0:'no'}

for column in yes_no_categorical_columns:
    creditCardData[column]=creditCardData[column].map(yes_no_values)

# creditCardData.marital_status = [1 if i == "Married" else 0 for i in creditCardData.marital_status]


In [11]:
 creditCardData.marital_status.value_counts()

,gender,car_owner,propert_owner,children,annual_income,type_income,education,marital_status,housing_type,birthday_count,employed_days,type_occupation,family_members,label
0,0,0,0,0,180000.0,Pensioner,Higher education,0,House / apartment,-18772.0,365243,NaN,2,1
1,0,0,0,0,315000.0,Commercial associate,Higher education,0,House / apartment,-13557.0,-586,NaN,2,1
2,0,0,0,0,315000.0,Commercial associate,Higher education,0,House / apartment,NaN,-586,NaN,2,1
3,0,0,0,0,NaN,Commercial associate,Higher education,0,House / apartment,-13557.0,-586,NaN,2,1
4,0,0,0,0,315000.0,Commercial associate,Higher education,0,House / apartment,-13557.0,-586,NaN,2,1


In [12]:
 creditCardData[''].value_counts() 

Working                 798
Commercial associate    365
Pensioner               269
State servant           116
Name: type_income, dtype: int64

In [13]:
creditCardData['housing_type'].value_counts()

House / apartment      1380
With parents             80
Municipal apartment      53
Rented apartment         21
Office apartment          9
Co-op apartment           5
Name: housing_type, dtype: int64

In [14]:
creditCardData['family_members'].value_counts()

2     802
1     334
3     268
4     127
5      15
6       1
15      1
Name: family_members, dtype: int64

In [20]:
creditCardData['education'].value_counts()

Secondary / secondary special    1031
Higher education                  426
Incomplete higher                  68
Lower secondary                    21
Academic degree                     2
Name: education, dtype: int64